官方文档：[https://www.tensorflow.org/get_started/mnist/pros?hl=zh-cn](https://www.tensorflow.org/get_started/mnist/pros?hl=zh-cn)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [20]:
# 导入数据
mnist = input_data.read_data_sets('/home/jimo/workspace/temp/mnist-data',one_hot=True)

Extracting /home/jimo/workspace/temp/mnist-data/train-images-idx3-ubyte.gz


Extracting /home/jimo/workspace/temp/mnist-data/train-labels-idx1-ubyte.gz
Extracting /home/jimo/workspace/temp/mnist-data/t10k-images-idx3-ubyte.gz
Extracting /home/jimo/workspace/temp/mnist-data/t10k-labels-idx1-ubyte.gz


In [21]:
# 定义变量
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

## 多层卷积网络

In [22]:
# 因为要多次创建权重和偏置变量，我们写个函数
def weights_variable(shape):
    # 生成服从正态分布且标准差为0.1的随机矩阵
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial_value=initial)

def bias_variable(shape):
    # 实践证明将bias设为很小的正数初识可避免死神经元
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial_value=initial)


In [23]:
# 卷积和池化
# 要理解这2个函数，参考：
def conv2d(x, W):
    # conv2d这个函数的理解参考：http://blog.csdn.net/mao_xiao_feng/article/details/78004522
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


In [34]:
# 第一层网络
# 5*5的小部分，1个图像通道，32个特征,也代表输出维数
W_conv1 = weights_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# x_image的参数：[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])
# 使用relu作为激励函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 图片大小减小到14*14
h_pool1 = max_pool_2x2(h_conv1)


In [35]:
# 第二层卷积层

# 将输入的32个特征映射成64个
W_conv2 = weights_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # 现在图片大小为7*7


In [36]:
# 全连接层

# 1024个神经元
W_fc1 = weights_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [37]:
# 为了避免过拟合，加入dropout

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


In [38]:
# 输出层

W_fc2 = weights_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [39]:
# 训练和评估模型

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
# 这次换种优化方式
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y_conv, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(200)
        if i % 100 == 1:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print("Step:%d accuracy:%g" % (i, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % sess.run(accuracy,
                                        feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


Step:1 accuracy:0.075


Step:101 accuracy:0.905


Step:201 accuracy:0.925


Step:301 accuracy:0.96


Step:401 accuracy:0.94


...

Step:1601 accuracy:0.97


Step:1701 accuracy:0.985


Step:1801 accuracy:0.99


Step:1901 accuracy:0.98


test accuracy 0.9856


可以看到其中也达到了0.995的准确度，平均下来还是不够高，官方训练了20000次，每次50张，达到0.992，但我这里就花了15分钟。